# Levenberg Marquardt

### Introduction: 

The [Levenberg Marquardt](https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm) algorithm a basic practical algorithm for solving [nonlinear least-sqaure](https://en.wikipedia.org/wiki/Non-linear_least_squares) problems. A better fitting name for this algorithm is damped gauss newton for it combines Gauss-Newton and Gradient Descent. If you have not already, please see my guides on [GuassNewton](GaussNewton.ipynb) and [Gradient Descent](GradientDescent.md) for this guide builds heavily off of concepts introduced in those algorithms. 

### Notation:

For clarity, this tutorial uses the following notation. Do not be alarmed if you are un-aware of any of the terms described below for they will be introduced in this tutorial. 

$$ m \ = \ number \ of \ samples  $$ 
$$ n \ = \ number \ of \ parameters $$ 
$$ \theta \ = \ model \ parameters $$ 
$$ r_{m,1} \ = \ residual \ = \ y - \hat y $$ 
$$ J_{m,n} \ = \ jacobian \ =  [ \nabla r ] $$ 
$$ p \ = \ step \ \  update \ \ to \ \  \theta  $$ 

### Theory: 

While [GuassNewton](GaussNewton.ipynb)(GN) is a basic nonlinear least squares(NLSQ) algorithm, it often fails to converge unless the initial conditions are very close to the correct solution. Likewise, [Gradient Descent](GradientDescent.md)(GD) can be another algorithm to solve NLSQ but it often takes far too many iterations to converge. Consequently, this algorithm attempts to blend those two algorithms together in order to achieve the following goals:  

* Robustness against a poor initial guesses
* Relatively quick convergence / low total iteration count

Practically speaking, Levenberg Marquardt is the most basic ** practical ** algorithm for solving NLSQ since the drawbacks of GN and GD often eliminate their application in industry. As mentioned previously, this algorithm combines GN and GD. Recall the GD and GN steps: 

$$ p_{GD} = \alpha \frac{J^T r}{ || J^T r ||} $$ 
$$ p_{GN} = -(J^T J)^{-1} J^T r  $$ 

The idea behind Levenberg Marquardt(LM) is to switch between these two steps based upon the performance of $r_k$ for each $p_k$ step. If the GN step reduce error($ || r || $) it is best to use it and if it increases the error, it is better to take a GD step. Rather than hard switching between these two algorithms, LM introduces a variable $\lambda$ that is heuristically updated based upon performance. The LM step is as follows: 

$$ p_{LM} = -(J^T J + \lambda I)^{-1} J^T r $$

The rational for this is if $\lambda$ is large, the term $ (J^T J + \lambda I)^{-1} $ simply becomes the identity matrix($I$) multiplied by some small scalar. This is equivalent of the GD step. Likewise, if $\lambda$ becomes 0, it is obviously the GN step. The next question then becomes how to update $\lambda$ ? This is done simply adjusting it based upon improvement in reduction in error. If the model error of a given iteration($k$) has decreased, reduce $\lambda_{k+1} = 0.1 \cdot \lambda_k $ and apply the step ($ \theta_{k+1} = \theta_k + p_k $). If the model error is increased,  increase lambda ($\lambda_{k+1} = 10 \cdot \lambda_k $) and do not apply the step ($ \theta_{k+1} = \theta_k $) . All together the algorithm can be summarized in the following steps: 

1. Calculate the residual: $r(\theta_k)$.
2. Calculate the jacobian: $J(\theta_k)$.
3. Calculate the LM step $p_k$
4. Calculate the new cost $r(\theta_k + p_k)$
5. IF ( $r(\theta_k + p_k)$ < $r(\theta_k)$): A, else B: 
   1. Decrease Lambda: ( $\lambda_{k+1} = 0.1 \cdot \lambda_k $). Update theta: ($ \theta_{k+1} = \theta_k + p_k $)
   2. Increase Lambda: ($\lambda_{k+1} = 10 \cdot \lambda_k$). Do not update theta:  ($ \theta_{k+1} = \theta_k $)
6. Determine if algorithm should terminate. 

Here is matlab/octave code to implement the above algorithm: 

```octave 
function theta = LMS(Rfnc,params,iterations)

alpha = 1;
theta = params;
oldCost = norm(Rfnc(theta));

for i =1:iterations;
    r = Rfnc(theta);
    J = Jf(Rfnc,theta);
    p = -pinv(J'*J + alpha*eye(length(params)))*J'*r;
    newCost = norm(Rfnc(theta+p));
    if(newCost<oldCost)
        theta = theta+p;  
        oldCost = newCost;
        alpha =0.1*alpha;
    else
        alpha = 10*alpha;
    end
end

end

```

In this algorithm, `` Rfnc `` is a function pointer to the residual function which should be $r(\theta) = y - f(\theta)$. ``Jf `` is a function to calculate the Jacobian. For simplicity in this case, the Jacobian is calculated through finite differences as opposed to the analytical derivative. Naturally this can be more computationally expensive. The code for ``Jf`` is below.
```octave
function J=Jf(fnc,params)

eps = 1e-8;
x1 = fnc(params);
m = size(x1,1);
n = length(params);
J = zeros(m,n);

for i = 1:n
    paramCpy = params; 
    paramCpy(i)= paramCpy(i) + eps;
    J(:,i) = (fnc(paramCpy) - x1)/eps;
end

end
```

### Example 1: 

As an example, consider a model of a sinusoid with unkown amplitude and frequency. 

$$ f(t) = \theta_1 sin( \frac{t}{ \theta_2}) $$ 

This model is conveniently two variables(for plotting purposes) as well as non-linear with respect to the parameters. This non-linearity is apparent when observing the jacobian.  

$$ \frac{\partial f}{\partial \theta} = \bigg[ sin(\frac{t}{ \theta_2}) \ , \ -\frac{\theta_1}{ \theta_2^2}  cos( \frac{t}{\theta_2})\bigg] $$

This non-linearity / non-convexity further observed in a plot of the cost function: 

<p align='center'><img src='Images/Nonlinear/GN_3.png'></p>

Previously, this problem was evaluated in my section on [GuassNewton](GaussNewton.ipynb) and it was stated that the initial guess ($\theta_0$) must be close to the solution in order for convergence. In contrast, the Levenberg Marquardt algorithm is much more robust. The following is an example with an initial guess of $\theta_0 = [100;100]$. 

<p align='center'><img src='Images/Nonlinear/LM_1.png'></p>

The following is the code. Note you will need to create the LMS function as the Jf function from the code provided above. 

```octave
clc
t = (-20:0.5:20)';
y = 5*sin(-t/5);
noise = 1*randn(length(t),1);
yM = y + noise; % This is the measurement. 
iterations = 50; % Fixed iteration length 
theta = [100;100];
rFncs = @(T) yM - (T(1)*sin(-t/T(2))); 

thetaGN = gaussNewtonNL(rFncs,theta,iterations);
thetaLM = LMS(rFncs,theta,iterations);

fig1 = figure(1);
clf(fig1);
hold on
scatter(t,yMeasured);
plot(t,y)
plot(t,thetaGN(1)*sin(-t/thetaGN(2)) )
plot(t,thetaLM(1)*sin(-t/thetaLM(2)) )

grid on
set(gca,'FontSize',10,'FontWeight','bold');
set(gcf,'Units','Pixels');
set(gcf, 'Position', [2500, 500, 750, 450]);
legend('Measured y', 'True y','Gauss Newton','Levenberg Marquardt')
title(['NLSQ: \theta_1 * sin(t / \theta_2)',' . \theta_0 = [',num2str(theta(1)),',',num2str(theta(2)),']'])
```

Of course, this single example does not prove much however it illustrates an example where LM converged where GN did not. To further illustrate this point, consider extending the initial guess through a range of values and take a statistical sampling of convergence. In the following example, $\theta_2$ is swept from -2000 to 3000. Each experiment was performed 100 times to ensure convergence was not due to noise.   

<p align='center'><img src='Images/Nonlinear/LM_2.png'></p>

The results rather painfully demonstrate the inadequacy of GN in practical application. In contrast, in this specific example, LM's initial guess of $\theta_2$ could be off by 3 orders of magnitude. In practice, how far off the initial guess can safely be will be dictated by the convexity of the cost plot. None the less, it demonstrates the potential reliability of LM if the problem is setup efficiently and a somewhat intelligent initial guess is made.

### Matlabs internal algorithm (2015b)

Another interesting point is that matlabs internal 'Levenberg Marquadrt' algorithm performs poorly in the face of badly scaled initial guesses. Their algorithm can be called via the following code:
```octave
 options.Algorithm = 'levenberg-marquardt';
 thetaM = lsqnonlin(rFncs,theta,[],[],options);
```

A while back I went through their code and I believe it is due to some initial parameter scaling they do. It may be related to how they calculate their jacobian but I'm assuming that is less likely. In my humble experience, the vanilla algorithm is the most robust.   

### Trust Region Method 'Improvement' 

The Trust Region method implements an 'improved' damped version of Gauss-Newton with a less heuristic method of updating $\lambda$. The trust region method updates $\lambda$ based upon the magnitude of $p_{k}$. The magnitude of $p_{k}$ is considered the trust region and this is what this algorithm heuristically updates. Arguably, this is a more intelligent approach than updating $\lambda$ arbitrarily based upon performance. This method was developed such that the algorithms steps operate within a region where the Taylor series quadratic approximation of the cost function is valid. Finding the appropriate $\lambda$ such that $||p_{k}||$ is within the trust region is known as the trust region sub problem. One way of stating this is the following: 

$$ p_k = (J^T J + \lambda I)^{-1} \quad s.t. \quad || p_k || < \Delta_k $$


Note, the sub problem itself can be solved fairly easily through newtons method since it is simply tunning a single parameter $\lambda$. However, using newtons method is not necessarily efficient or always stable. Consequently, many methods have been proposed for solving the trust region method. [Numerical Optimization](https://www.amazon.com/Numerical-Optimization-Operations-Financial-Engineering/dp/0387303030) is a defacto reference in trust region methods and possible solutions. The method I implemented was based off this and [nmayorov](https://nmayorov.wordpress.com/2015/06/18/basic-algorithms-for-nonlinear-least-squares/). The benefit and goal of this method is to not to have to recalculate the SVD multiple times.


$$ SVD(J) = U \Sigma V^T \\ \phi = || \frac{\Sigma U r}{ \Sigma^2 + \lambda}|| - \Delta \\  \phi^{\prime} = - sum \bigg( \frac{ \big( \Sigma U r \big)^2}{ \big( \Sigma^2 + \lambda \big)^3 } \cdot (1 / || \frac{\Sigma U r}{ \Sigma^2 + \lambda}|| \bigg) $$

At which point $ \alpha $ can be heuristically calculated with:

$$ \lambda_{k+1} = \lambda_{k} - \frac{ \phi(\lambda_k)}{\phi^{\prime}(\lambda_k)} $$


After acouple iterations, $\lambda$ is within the desired trust region. This algorithm itself is fairly complicated and i'll leave it up to Numerical Optimization to prove this method. While it is argued to be better, in my limited experience, I did not see significant improvements from using this algorithm. Granted, an exhaustive set of test vectors should be used to validate a given optimization algorithm and its performance.   

<p align='center'><img src='Images/Nonlinear/TrustRegion.png'></p>

Perhaps this algorithm would see more benefit in large large scale problems but for basic application, vanilla damped Gauss-Newton is probably the route to go. However, I could have acourse made a mistake in my implementation. My implementation is as follows:  

```octave
function theta = lmaT(x,fnc,params)

alpha = 1;
theta = params;
Delta = norm(params);
r = fnc(x,theta);
oldCost = 0.5*(r' * r);
J = jac(x,fnc,theta);
g = (J')*r; 
maxEvals = 100;

for i =1:maxEvals;

    [U,S,V] = svd(J,'econ');
    S = diag(S);
    actualR = -1;
    while ((i < 100) && (actualR <= 0))
        [p,alpha] = solveTrustRegion(U,S,V,alpha,Delta,r);
        pNorm = norm(p);
        predR = -calcQuad(J,g,p);
        
        thetaNew = theta + p;
        rNew = fnc(x,thetaNew);
        newCost = 0.5*((rNew') * rNew);
        actualR = oldCost - newCost;
        
        %Update Trust Region 
        if predR > 0
            ratio = actualR / predR;
        else
            ratio = 0;
        end
        
        if ratio < 0.25
            Delta = 0.25 * pNorm;
        elseif (ratio > 0.75) && ( pNorm > 0.95 * Delta)
            Delta = Delta * 2;
        end
        i = i+1;      
    end
    
    terminate = checkTermination(pNorm, norm(thetaNew));
    if(terminate)
        break;
    end
    
    if (actualR > 0)
        theta = thetaNew;
        oldCost = newCost;
        r = rNew;
        J = jac(x,fnc,theta);
        g = J' * r; 
    end 
    
end

end

function [phi,phi_prime] = calcPhi(alpha,sur,s,Delta)
    denom = s.^2 + alpha;
    p_norm = norm(sur ./ denom);
    phi = p_norm - Delta;
    phi_prime = -1*sum(sum((sur.^2)./(denom.^(3))))./ p_norm;
end

function [p,alpha]= solveTrustRegion(u,s,v,alpha,Delta,r)
    
    ur = (u')*r;
    sur = s.*ur;
    
    p = -v*(ur ./ s);
    if(norm(p) <= Delta) % Are we within the trust region? 
        return; % We are done 
    end
    alpha_u = norm(sur) / Delta; % Upper alpha limit 
    alpha_l = 0; % Lower alpha limit 
    if(alpha==0)
        alpha = max(0.001*alpha_u, (alpha_u * alpha_l)^(1/2));
    end
    
    for i = 1:10
        if((alpha < alpha_l) || (alpha> alpha_u))
            alpha = max(0.001*alpha_u, (alpha_u * alpha_l)^(1/2));
        end
        [phi,phi_prime] = calcPhi(alpha,sur,s,Delta);
        if phi < 0
            alpha_u = alpha;
        end
      
        phiR = phi / phi_prime;
        alpha_l = max(alpha_l, alpha - phiR);

        alpha = alpha-((phi + Delta) * phiR / Delta);
        
        if abs(phi) < 0.001 * Delta % is phi near 0; 
            break;
        end   
    end
    
    p = -v * (sur ./ (s.^2 + alpha)); % final solution for step
    p = p * Delta / norm(p); % Ensure in trust region
end

function q = calcQuad(J,g,p)
    q = 0.5 *(p') * ((J') * J) * p + (g')*p;
end

function term = checkTermination(stepNorm,thetaNorm)
    term = false; 
    if stepNorm < ( 1e-5 * (1e-5 + thetaNorm)) % If the step size is tiny compared to the norm of the parameters 
        term = true;
    end
end

```


### Conclusion: 

The Levenberg Marquardt is an effective and simple method of solving nonlinear least squares for a local optimal solution. Whether that solution is the true optimal depends highly on the problem itself as well as the accuracy of the initial guess. With careful consideration, LMA can be implemented in practice if a reliable system of initial guesses is devised and the problem is well suited for NLSQ. In addition to LMA there is a trust region approach which is arguably an improvement. However, in my limited experience, LMA is far more straight forward to implement and in a decent amount of simple cases, simply perform betters. 


